In [151]:
import pandas as pd
import numpy as np
from itertools import combinations 
import networkx as nx
import nxviz as nv

In [2]:
title_akas = pd.read_csv('title.akas.tsv', sep='\t', index_col = 'titleId', usecols=['titleId', 'language','title', 'isOriginalTitle'], low_memory=False)
req = (set(title_akas[(title_akas['language']=='en')].index))
title_basics = pd.read_csv('title.basics.tsv', sep='\t', low_memory=False, 
                           usecols=['tconst','titleType', 'primaryTitle', 'genres'], index_col= 'tconst')



In [3]:
#Featurize genres
node_data = title_basics.reindex(req)
node_data['genres'] = node_data['genres'].str.split(',')   #Convert genres to lists
node_data.dropna(inplace=True)   #Remove the one row with no data
genres_edge = pd.Series(node_data['genres']).apply(frozenset).to_frame(name='genre')
for genre in frozenset.union(*genres_edge.genre):
    genres_edge[genre] = genres_edge.apply(lambda _: int(genre in _.genre), axis=1)

In [78]:
genres_for_edges = ['Sci-Fi', 'Mystery']
num=0
edges=set()
for x in genres_for_edges:
    print (x, "- Movie Count: " ,sum(genres_edge[x]))
    for x,y in combinations(genres_edge[genres_edge[x]==1].index, 2):
        num += 1
        edges.add((x,y))

num_combinations = num #Number of possible combinations explored
num_unique_edges = len(edges) #Number of unique edges
print (num_combinations - num_unique_edges)
print (num_unique_edges)
a = len(node_data.index)
print (a * (a - 1)) #Max number of edges possible in the graph

Sci-Fi - Movie Count:  2401
Mystery - Movie Count:  3424
9180
8732196
13135337490


In [140]:
node_data.head()  #Node data

,titleType,primaryTitle,genres
tconst,,,
tt0817928,movie,Laa-thaa-phii,[Horror]
tt0095493,movie,Last Platoon,"[Action, War]"
tt0345124,movie,The Sorrowful River,[Drama]
tt1555385,short,C@ught in the Net,"[Drama, Short]"
tt0067547,movie,Memory,"[Documentary, War]"


In [84]:
list(edges)[:5] #Edge data

[('tt0262614', 'tt0245013'),
 ('tt0334347', 'tt0075462'),
 ('tt1160629', 'tt4985818'),
 ('tt5626536', 'tt0174975'),
 ('tt0093890', 'tt5564148')]

In [175]:
#Begin Graph

G = nx.Graph()
G.clear()
for index,row in node_data.iterrows():
    G.add_node(index, data=row['primaryTitle'])
for x in list(edges):
    G.add_edge (x[0],x[1])